In [ ]:
import random
import sys

import autograd.numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.signal import convolve
import scipy.stats

In [ ]:
sys.path.append("..")

import shared.format
import shared.tools

In [ ]:
SEED = 14

np.random.seed(SEED)
random.seed(SEED)

In [ ]:
import autocrit.utils.random_matrix

In [ ]:
plt.rcParams["font.size"] = shared.format.FONTSIZE

In [ ]:
wigner = autocrit.utils.random_matrix.SymmetricWigner(50)

In [ ]:
spectrum = wigner.eigvals()

In [ ]:
precision = 1e-4

In [ ]:
lams = wigner.generate_lams(wigner.max_lam + precision, precision)
expected_spectrum = [wigner.expected_spectral_density(lam) for lam in lams]

expected_csds = wigner.expected_cumulative_spectral_distribution(
    wigner.max_lam + precision, precision, accumulate=True)

In [ ]:
fig = plt.figure(figsize=(12, 4))
spec = gridspec.GridSpec(ncols=3, nrows=1, figure=fig)
mat_ax = fig.add_subplot(spec[0, 0])
spec_ax = fig.add_subplot(spec[0, 1])
cdf_ax = fig.add_subplot(spec[0, 2])

mat_ax.imshow(wigner.M, cmap="Greys")
mat_ax.axis("off")

spec_ax.hist(spectrum, density=True, histtype="step",
             lw=shared.format.LINEWIDTH,
             label="Empirical")
spec_ax.plot(lams, expected_spectrum,
             lw=shared.format.LINEWIDTH, linestyle="--",
             label="Semicircular")
spec_ax.set_ylabel(r"$\rho$", fontdict={"size": "large"})
spec_ax.set_xlabel(r"$\lambda$", fontdict={"size": "large"})

cdf_ax.step(np.array([-2] + list(spectrum) + [2]),
            np.divide([0] + list(range(len(spectrum))) + [len(spectrum)], len(spectrum)),
            lw=shared.format.LINEWIDTH)
cdf_ax.plot(lams[1:], expected_csds,
            lw=shared.format.LINEWIDTH, linestyle="--")
cdf_ax.set_ylabel(r"$\rho(\lambda \leq \Lambda)$", fontdict={"size": "large"})
cdf_ax.set_xlabel(r"$\lambda$", fontdict={"size": "large"})

fig.tight_layout()
spec_ax.legend(loc=[0.45, -0.4], ncol=2);

shared.tools.add_panel_label("A", mat_ax)
shared.tools.add_panel_label("B", spec_ax)
shared.tools.add_panel_label("C", cdf_ax)

bbox = fig.bbox_inches.from_bounds(-0.25, -0.5, 12.25, 5.)
fig.savefig("wigner-rm.pdf", bbox_inches=bbox)